# Cogs 108 Final Project 
Allison Reiss, Brendan Taing, Evan Barosay, Gael Van der Lee, Daniel Benamou, Adham Rafiq

The following data analysis is centered around the following research question:

Can we distinguish the safety of different neighborhoods in San Diego based on the number of calls dispatched by the SDPD based on the degree of the call and number of dispatches, and if so, are they related to income and population?

## Import and Clean Data

In [142]:
import pandas as pd
import numpy as np
import operator
from operator import itemgetter


#load data into dataframes
beat = pd.read_csv('pd_beat_neighborhoods_datasd.csv')
call = pd.read_csv('pd_calls_for_service_2017_datasd.csv')
census = pd.read_csv('sandiegocensustract.csv')

In [143]:
# This is cleaning up the call df columns
call = call[['date_time','street','streettype','beat', 'priority']]


# This is cleaning up the census df to drop unneccessary columns
census = census.drop('TRACTNUM', axis = 1)
census = census.drop('TRACT', axis = 1)
census = census.drop('SevCrwd', axis = 1)
census = census.drop('TeenBirthAllWom', axis = 1)
census = census.drop('MaltrtAllegRate', axis = 1)
census = census.drop('ProxOffAlco', axis = 1)
census = census.drop('ProxOnAlco', axis = 1)
census = census.drop('TraffInjur', axis = 1)
census = census.drop('SNAP_FdStmp', axis = 1)
census = census.drop('n_FoodDesert', axis = 1)
census = census.drop('pct_FoodDesert', axis = 1)
census = census.drop('LiqCount', axis = 1)
census = census.drop('VoterPartic', axis = 1)
census = census.drop('HghSchOrHigh', axis = 1)
census = census.drop('Uninsured', axis = 1)
census = census.drop('PovertyPctl', axis = 1)
census = census.drop('Longitude', axis = 1)
census = census.drop('Latitude', axis = 1)
census = census.drop('LowBirthWeight', axis = 1)
census = census.drop('LowBirthWeightPctl', axis = 1)
census = census.drop('PollutionBurdenPctl', axis = 1)
census = census.drop('CES20Score', axis = 1)
census = census.drop('SingMother', axis = 1)
census = census.drop('TeenBirthProportion', axis = 1)
census = census.drop('PollutionBurdenScore', axis = 1)
census = census.drop('CES20PercentileRange', axis = 1)
census = census.drop('PopCharScore', axis = 1)
census = census.drop('PopCharPctl', axis = 1)
census = census.drop('FosterCareEntry', axis = 1)
census = census.drop('TotalPov', axis = 1)
census = census.drop('ChildPov', axis = 1)
census = census.drop('Education', axis = 1)

# change names of columns in beat dataframe to be compatible for merge
beat.columns = [
    'beat', 'neighborhood'
]

In [144]:
# Replace date_time column to include only year
call['date_time'] = call['date_time'].str[:4]
call.columns = ['year','street','streettype','beat', 'priority']

In [145]:
# Replace all 0s and 999s in beat with NaN. 
call['beat'] = call['beat'].replace(0,'NaN')
call['beat'] = call['beat'].replace(999,'NaN')

# Drop all rows with null values
call = call.dropna(how='any')

In [146]:
# Merge the call dataframe with beat dataframe to replace beat code with actual neighborhood
call = pd.merge(call, beat, on='beat')

call = call.drop(['beat'], axis=1)

## Analyze the Call dataframe and create a Heatmap to show dangerous areas

In [147]:
### PROBABLY DELETE THIS LATER?? I don't think we need it anymore lol

#  Function Name: get_threat_lvl
#  Purpose: to find the "threat level" of a neighborhoof
#          Threat level = sum of (priority of call*number of calls) for each neighborhood (can later do street)
#  Parameter(s): df (Dataframe containing all of the calls/priorities for a certain neighborhood)
#  Returns: The threat level of the neighborhood (float)
#
# Example Usage: 
#   get_threat_lvl(PB_df) could return 3.5 (idk if thats the actual value lol just an example)

def get_threat_lvl(df):
    # get the counts for each number of priority call in that neighborhood
    sum_1 = np.count_nonzero(df['priority'] == 1.0)
    sum_2 = np.count_nonzero(df['priority'] == 2.0)
    sum_3 = np.count_nonzero(df['priority'] == 3.0)
    sum_4 = np.count_nonzero(df['priority'] == 4.0)
    
    threat_lvl = (sum_1*1.0) + (sum_2*2.0) + (sum_3*3.0) + (sum_4*4.0)
    
    return threat_lvl

In [148]:
# Get list of unique neighborhoods in dataframe
unique_neighborhoods = call['neighborhood'].unique()

In [149]:
# This is returning a series of the neighborhoods with the most calls from greatest to least
# We might be able to show a heat map of this later
neighborhoodcalls = call['neighborhood'].value_counts()

In [150]:
# Loop through list and run algorithm on each neighborhood to get the safety level of each neighborhood
# and add it to a dict -- (neighborhood, safety level) pair

#create a dict to store the priority call sums per neighborhood
sketchy_lvl_dict = {}

#idx is the neighborhood name, i is the index
for idx,i in zip(call['neighborhood'],range(call['neighborhood'].size)):
    if idx in sketchy_lvl_dict:
        sketchy_lvl_dict[idx] = sketchy_lvl_dict[idx] + call['priority'][i]
    else:
        sketchy_lvl_dict[idx] = call['priority'][i]

In [151]:
# sort the dict to priority call sums in order of least to greatest
sorted_sketchy = sorted(sketchy_lvl_dict.items(), key=operator.itemgetter(1))
max_sketchiness = max(sorted_safety,key=itemgetter(1))[1]

In [152]:
# Get the comparative percentages by dividing by highest sketchiness value to make data easier to read
for key in sketchy_lvl_dict:
    sketchy_lvl_dict[key] = (sketchy_lvl_dict[key] / max_sketchiness) * 100

In [153]:
# sort the dict of percentages
sketchy_percents = sorted(sketchy_lvl_dict.items(), key=operator.itemgetter(1))
sketchy_percents

[('Islenair', 0.51895931358981451),
 ('Tijuana River Valley', 0.62275117630777743),
 ('Burlingame', 0.69194575145308612),
 ('Broadway Heights', 0.69194575145308612),
 ('Rancho Encantada', 1.3631331303625795),
 ('Qualcomm', 1.5707168557985054),
 ('San Pasqual', 1.6952670910600609),
 ('Alta Vista', 1.8959313589814559),
 ('Torrey Highlands', 2.3802933849986161),
 ('North City', 2.9200110711320235),
 ('La Playa', 3.0168834763354551),
 ('Torrey Pines', 3.3974536396346524),
 ('Black Mountain Ranch', 3.6050373650705785),
 ('Wooded Area', 3.7849432604483804),
 ('Fox Canyon', 4.5737614171048993),
 ('Fairmount Park', 4.6429559922502079),
 ('Harborview', 5.5701632991973424),
 ('Sunset Cliffs', 5.6255189593135899),
 ('Miramar Ranch North', 5.7500691945751452),
 ('Adams North', 5.9092167174093557),
 ('Rolando Park', 6.1029615278162188),
 ('Shelltown', 6.123719900359812),
 ('Redwood Village', 6.2067533905341827),
 ('Azalea/Hollywood Park', 6.2344312205923051),
 ('Swan Canyon', 6.324384168281207),
 (

In [154]:
# upload Google API for HeatMaps

In [155]:
# Figure out how to make a heat map

## The section below will start to analyze the correlations between income and other demographic variables 

In [156]:
# We will fill in these steps after completing the first part of our analysis (the heat maps)

In [157]:
#looking at unique neighborhood names, probably delete later but to visualize data
print (census['NeighborhoodName'].unique().size)
census['City'].unique()

238


array(['San Diego', 'San Marcos', 'Oceanside', 'Escondido', 'Vista',
       'Carlsbad', 'El Cajon', 'Fallbrook', 'National City', 'La Mesa',
       'San Ysidro', 'Encinitas', 'Lakeside', 'Chula Vista',
       'Borrego Springs', 'Spring Valley', 'Poway', 'Santee',
       'Imperial Beach', 'Campo', 'Warner Springs', 'Solana Beach',
       'Lemon Grove', 'La Jolla', 'Valley Center', 'Ramona',
       'Pauma Valley', 'Bonita', 'Coronado', 'Alpine', 'Descanso', 'Jamul',
       'Cardiff By the Sea', 'Del Mar', 'Bonsall', 'Julian',
       'Rancho Santa Fe', 'Camp Pendleton', nan], dtype=object)